In [1]:
import pandas as pd
import json

In [ ]:
data = []
with open('./tmdbKr.json','r',encoding='utf8') as file :
    data= json.load(file)

In [ ]:
# 빈 데이터프레임
df = pd.DataFrame(columns=['title','genres','id','overview','popularity'])

In [ ]:
# 영화데이터

# 모든데이터 for문 돌면서
# model == movies.movie 일때
# 각 값들을 뽑아서 새로운 데이터프레임으로 만들고 
# 기존 데이터프레임에 합쳐주는 과정
for i in range(len(data)) :
    if data[i]['model']=='movies.movie':
        title =  data[i]['fields']['title']
        gen = data[i]['fields']['genre_ids']
        gen_str = ','.join(str(e) for e in gen) # 여러값이 있어서 해야하는데 이해안감
        id = data[i]['pk']
        overview = data[i]['fields']['overview']
        popularity = data[i]['fields']['popularity']

        data_json = {'title':[title],'genres':[gen_str],'id':[id],'overview':[overview],'popularity':[popularity]}
        df_tmp= pd.DataFrame(data_json)
        df =  pd.concat([df, df_tmp], ignore_index = True)

In [ ]:
df.to_csv('./영화데이터.csv',encoding='utf8',index=False)

## 영화 검색

In [ ]:
from tmdbv3api import TMDb, Movie

tmdb = TMDb()
tmdb.api_key = "234c446afa7c9156e6c9a101b724f5ae"

movie = Movie()

m = movie.details(111)

print(m.title)
print(m.overview)
print(m.popularity)


Scarface
After getting a green card in exchange for assassinating a Cuban government official, Tony Montana stakes a claim on the drug trade in Miami. Viciously murdering anyone who stands in his way, Tony eventually becomes the biggest drug lord in the state, controlling nearly all the cocaine that comes through Miami. But increased pressure from the police, wars with Colombian drug cartels and his own drug-fueled paranoia serve to fuel the flames of his eventual downfall.
83.822


In [ ]:
# 영화 검색
import requests


def SearchMovies(query, year):  # 영화 제목과 년도 정보를 받음

    params = {"api_key": "234c446afa7c9156e6c9a101b724f5ae",
          "language": "ko",
          "page": "1",
          "query": query,
          "include_adult": "true",
          "year": year}


    url = "https://api.themoviedb.org/3/search/movie" # 정보를 요청할 주소
    resp = requests.get(url, params=params)
    data = resp.json()['results']


    if(len(data) > 0): # 검색해서 나온 데이터가 있을 경우에만

       for item in data : # 반복문 돌면서 영화 이름과 제작년도 출력
           print(item['original_title'], item['release_date']) 

if __name__ == '__main__':
    SearchMovies('존 윅',"2023")

John Wick: Chapter 4 2023-03-22
